In [62]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    patchelf \
    xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglew-dev is already the newest version (2.0.0-5).
patchelf is already the newest version (0.9-1).
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
libosmesa6-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
software-properties-common is already the newest version (0.96.24.32.18).
xvfb is already the newest version (2:1.19.6-1ubuntu4.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [63]:
!pip install gym==0.21.0
!pip install free-mujoco-py
!pip install transformers
!pip install datasets
!pip install imageio-ffmpeg

!pip install colabgymrender==1.0.2
!pip install xvfbwrapper
!pip install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 33.3 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imageio-2.4.1-py3-none-any.whl
  Attempting uninstall: imageio
    Found existing installation: imageio 2.21.3
    Uninstalling imageio-2.21.3:
      Successfully uninstalled imageio-2.21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
free-mujoco-py 2.1.6 requi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import os
import random
from tqdm import tqdm

from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn

from datasets import load_dataset

In [93]:
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "walker2d-expert-v2")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset decision_transformer_gym_replay downloaded and prepared to /root/.cache/huggingface/datasets/edbeeching___decision_transformer_gym_replay/walker2d-expert-v2/1.1.0/8a8520881dd09333b04f5a116dd47931314ec44b72597bce8675d40ccd42ad88. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [94]:
@dataclass
class DataLoader:
    def __init__(self, dataset, gamma=1.0, epsilon=1e-5):
        self.dataset = dataset
        self.epsilon = epsilon
        self.gamma = gamma

        self.states = []
        trajectories_length = []
        for obs in self.dataset["observations"]:
            self.states.append(obs)
            trajectories_length.append(len(obs))
        self.states = np.vstack(self.states)
        self.length = self.states.shape[0]

        mean = np.mean(self.states, axis=0)
        std = np.std(self.states, axis=0) + self.epsilon
        self.states = (self.states - mean) / std

        self.actions = []
        for acts in self.dataset["actions"]:
            self.actions.append(acts)
        self.actions = np.vstack(self.actions)


    def __call__(self, batch_size):
        batch_idxs = np.random.choice(
            self.length,
            size=batch_size,
            replace=True,
        )

        s = self.states[batch_idxs]
        a = self.actions[batch_idxs]
        s = torch.from_numpy(np.vstack(s)).float()
        a = torch.from_numpy(np.vstack(a)).float()

        return {
            "states": s,
            "actions": a,
        }

In [95]:
dl = DataLoader(dataset["train"])

In [96]:
dl(2)

{'states': tensor([[-7.4475e-02,  2.6103e-01,  7.0889e-01,  9.1304e-01,  7.9653e-01,
           1.7768e-01, -8.9549e-01, -2.0794e+00,  2.9730e-01,  3.9217e-01,
          -4.6746e-01,  9.5046e-02,  2.0260e-02,  6.2291e-02, -5.8059e-01,
          -9.7654e-01,  9.4182e-01],
         [ 1.0113e+00, -2.9177e-01, -8.6516e-01, -8.0092e-01, -1.2084e+00,
           4.7180e-01,  1.0714e+00,  7.8032e-01,  3.6431e-01, -1.3979e+00,
          -4.8761e-01, -1.0822e+00,  2.9062e-01,  1.9519e+00,  1.0835e-01,
          -5.3739e-02,  2.9720e-04]]),
 'actions': tensor([[ 0.1098,  0.1122,  0.9863,  0.9705,  0.4205, -0.0804],
         [ 0.7045,  0.6313,  0.3737,  0.9394,  0.7006,  0.9789]])}

In [97]:
class MLPBehaviorCloning(nn.Module):
    """
    Simple MLP that predicts next action a from past states s.
    """

    def __init__(self, state_size, action_size, hidden_size, n_layer, dropout=0.1, **kwargs):
        super().__init__()

        self.hidden_size = hidden_size

        layers = [nn.Linear(state_size, hidden_size)]
        for _ in range(n_layer-1):
            layers.extend([
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, hidden_size)
            ])
        layers.extend([
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, action_size),
            nn.Tanh(),
        ])

        self.model = nn.Sequential(*layers)


    def forward(self, states):
        actions = self.model(states)
        return actions


    def forward_loss(self, states, actions):
        action_preds = self.forward(states)

        loss = torch.mean((action_preds - actions) ** 2)
        return loss

In [98]:
env = gym.make("Walker2d-v2")

state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

In [99]:
model = MLPBehaviorCloning(state_size, action_size, 256, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [106]:
epochs = 2000
batch_size = 64
num_iters = (len(dataset["train"])-1) // 64 + 1

In [107]:
pbar = tqdm(range(epochs))

for e in pbar:
    sum_loss = 0.0
    for i in range(num_iters):
        data = dl(batch_size)
        loss = model.forward_loss(**data)

        optimizer.zero_grad()        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()

        sum_loss += loss.item()

    pbar.set_postfix(loss=sum_loss)

100%|██████████| 2000/2000 [03:06<00:00, 10.71it/s, loss=0.582]


## Eval


In [108]:
import mujoco_py
import gym

from colabgymrender.recorder import Recorder

In [109]:
env = gym.make("Walker2d-v2")
env = Recorder(env, ".", fps=30)

device = "cpu"
max_ep_len = 10000
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

In [112]:
state = env.reset()

for t in range(max_ep_len):
    with torch.no_grad():
        state = torch.from_numpy(state).reshape(1, state.shape[0]).to(device=device, dtype=torch.float32)
        action = model(state)[0].numpy()
        
    state, reward, done, _ = env.step(action)

    if done:
        break


In [113]:
# Play the video
env.play()
# If you want to convert the video:
# !ffmpeg -i {your_video} -vcodec h264 replay.mp4 

100%|██████████| 118/118 [00:00<00:00, 184.07it/s]
